## Setup

In [1]:
import json
import os
import openai   

# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)
    
# Setting up the deployment name
deployment_name = config_details['DEPLOYMENT_NAME']

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = config_details['OPENAI_API_BASE']

# Currently Chat Completion API have the following versions available: 2023-07-01-preview
openai.api_version = config_details['OPENAI_API_VERSION']

## Define the function(s)

In [27]:
def get_cheapest_flight_price(start, destination, date, features):
    # Your code here
    return "1200"

available_functions = {
            "get_cheapest_flight_price": get_cheapest_flight_price,
            #"get_stock_market_data": get_stock_market_data,
            #"calculator": calculator,
        } 

## Prepare user query and function details for the model

In [28]:
messages = [
    {"role": "user", "content": "What is the cheapest non-stop flight from Seattle to London on August 25, 2023?" }
]

functions= [  
    {
        "name": "get_cheapest_flight_price",
        "description": "Retrieves flight information based on the parameters provided",
        "parameters": {
            "type": "object",
            "properties": {
                "start": {
                    "type": "string",
                    "description": "The starting location (i.e. Seattle)"
                },
                "destination": {
                    "type": "number",
                    "description": "The destination location"
                },
                 "date": {
                    "type": "string",
                    "description": "The date of the flight, e.g. March 1, 2024."
                },
                "features": {
                    "type": "string",
                    "description": "A comma separated list of features (i.e. cheap, non-stop, etc.)"
                }
            },
            "required": ["start"],
            "required": ["destination"],
            "required": ["date"],
            "required": ["features"]
        },
    }
]  

## Submit user message and desired function(s) definitions to the model

In [29]:
response = openai.ChatCompletion.create(
    deployment_id=deployment_name,
    messages=messages,
    functions=functions,
    function_call="auto", 
)

## Output the recognized function call and parameters

In [30]:
print(response['choices'][0]['message'])

{
  "role": "assistant",
  "function_call": {
    "name": "get_cheapest_flight_price",
    "arguments": "{\n  \"start\": \"Seattle\",\n  \"destination\": \"London\",\n  \"date\": \"August 25, 2023\",\n  \"features\": \"non-stop,cheap\"\n}"
  }
}


## Validate the recognize function call exists

In [31]:
function_name = response['choices'][0]['message']["function_call"]["name"]
        
# verify function exists
if function_name not in available_functions:
    print ("Function " + function_name + " does not exist.")
else:
    print ("Function " + function_name + " exists.")

Function get_cheapest_flight_price exists.


## Call the function to fetch the information

In [33]:
function_to_call = available_functions[function_name]
function_args = json.loads(response['choices'][0]['message']["function_call"]["arguments"])
function_response = function_to_call(**function_args)
        
print("Output of function call:")
print(function_response)
print()

Output of function call:
1200



## Submit the function output to the model

In [34]:
# adding function response to messages
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": function_response
    }
)  # extend conversation with function response

second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_name,
        )  # get a new response from GPT where it can see the function response


## Print the model's response to the user

In [37]:
print (second_response["choices"][0]["message"]["content"])

The cheapest non-stop flight from Seattle to London on August 25, 2023, is priced at $1200.
